In [1]:
import concurrent.futures
from tqdm import tqdm
import os
import pandas as pd
import time
import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, WebDriverException

def scrape_weather_to_df(name):
    # Set up options để giảm tài nguyên sử dụng
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--disable-extensions')
    options.add_argument('--disable-infobars')
    options.add_argument('--disable-notifications')
    options.add_argument('--blink-settings=imagesEnabled=false')  # Tắt tải hình ảnh
    
    driver = webdriver.Chrome(options=options)
    url = f"https://www.worldweatheronline.com/{name}-weather-history/vn.aspx"
    driver.get(url)
    
    data = []
    try:
        allow_cookies = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.ID, "CybotCookiebotDialogBodyButtonAccept"))
        )
        allow_cookies.click()
        time.sleep(1)
    except (NoSuchElementException, TimeoutException):
        pass
    
    record_keys = ['Time', 'Weather', 'Temp', 'Rain', 'Cloud', 'Pressure', 'Wind', 'Gust']
    date = datetime.datetime(2010, 1, 1)
    end_date = datetime.datetime(2025, 4, 5)
    
    try:
        while date < end_date:
            date_str = date.strftime('%Y-%m-%d')
            try:
                input_date = WebDriverWait(driver, 7).until(
                    EC.presence_of_element_located((By.ID, 'ctl00_MainContentHolder_txtPastDate'))
                )
                driver.execute_script("arguments[0].value = arguments[1];", input_date, date_str)
                submit_date = driver.find_element(By.ID, 'ctl00_MainContentHolder_butShowPastWeather')
                submit_date.click()
            except WebDriverException:
                date += datetime.timedelta(days=1)
                continue
            
            time.sleep(1)
            
            tables = driver.find_element(By.XPATH, "/html/body/form/div[3]/section/div/div/div/div[3]/div[1]/div/div[3]/table/tbody")
    # lấy lại phần tử sau khi trang đã reload
            all_rows = tables.find_elements(By.TAG_NAME, "tr")
            rows = all_rows[2:10] 
            
            for row in rows:
                try:
                    cells = row.find_elements(By.CLASS_NAME, "days-details-row-item1")
                    rains = row.find_elements(By.CLASS_NAME, "days-rain-number")
                    rain = rains[0].text
                    weather_img = cells[1].find_element(By.TAG_NAME, "img")
                    weather = weather_img.get_attribute("title")
                    
                    values = [cells[0].text.strip(), weather, cells[2].text.strip(), rain, cells[3].text.strip(), 
                            cells[4].text.strip(), cells[5].text.strip(), cells[6].text.strip()]
                    
                    if values:
                        data.append([date_str] + values)
                except Exception:
                    continue
            
            date += datetime.timedelta(days=1)
    finally:
        driver.quit()
    
    if data:
        df = pd.DataFrame(data, columns=["Date"] + record_keys)
        return df
    else:
        return pd.DataFrame(columns=["Date"] + record_keys)

def process_location(name, output_dir):
    try:
        print(f"Đang crawl dữ liệu từ {name}")
        df = scrape_weather_to_df(name)
        filename = os.path.join(output_dir, f"{name}.csv")
        df.to_csv(filename, index=False)
        return f"✅ Đã lưu dữ liệu {name} vào {filename}", name, True
    except Exception as e:
        return f"❌ Lỗi khi crawl {name}: {e}", name, False

def main():
    output_dir = r"E:\Tài liệu học\2024-2\Lưu trữ và xử lý dữ liệu lớn\Data"
    os.makedirs(output_dir, exist_ok=True)
    
    name_tinh_process = ['thanh-hoa'
                         , 'tra-vinh'
                         , 'tuy-hoa'
                         , 'tuyen-quang'
                         , 'uong-bi'
                         , 'viet-tri'
                         , 'vinh'
                         , 'vinh-long'
                         , 'vinh-yen'
                         , 'vung-tau'
                         , 'x.-ba-luong'
            ]  # Thay bằng danh sách của bạn
    
    # Số luồng chạy đồng thời (tùy chỉnh theo cấu hình máy)
    max_workers = 4  # Nên để số lượng nhỏ để tránh bị chặn
    
    results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Gửi các tác vụ
        future_to_name = {
            executor.submit(process_location, name, output_dir): name 
            for name in name_tinh_process
        }
        
        # Hiển thị tiến trình
        with tqdm(total=len(name_tinh_process), desc="🌤️ Crawling weather data") as pbar:
            for future in concurrent.futures.as_completed(future_to_name):
                name = future_to_name[future]
                try:
                    message, location, success = future.result()
                    print(message)
                    results.append((location, success))
                except Exception as e:
                    print(f"❌ Lỗi với {name}: {e}")
                    results.append((name, False))
                pbar.update(1)
    
    # Thống kê kết quả
    success_count = sum(1 for _, success in results if success)
    print(f"\n✅ Đã crawl thành công: {success_count}/{len(name_tinh_process)}")
    
    if success_count < len(name_tinh_process):
        failed_locations = [name for name, success in results if not success]
        print(f"❌ Các địa điểm chưa crawl được: {', '.join(failed_locations)}")

if __name__ == "__main__":
    main()

Đang crawl dữ liệu từ thanh-hoa
Đang crawl dữ liệu từ tra-vinh
Đang crawl dữ liệu từ tuy-hoa
Đang crawl dữ liệu từ tuyen-quang


🌤️ Crawling weather data:   9%|▉         | 1/11 [3:01:14<30:12:25, 10874.59s/it]

Đang crawl dữ liệu từ uong-bi
✅ Đã lưu dữ liệu tuyen-quang vào E:\Tài liệu học\2024-2\Lưu trữ và xử lý dữ liệu lớn\Data\tuyen-quang.csv


🌤️ Crawling weather data:  18%|█▊        | 2/11 [4:09:28<17:12:54, 6886.01s/it] 

Đang crawl dữ liệu từ viet-tri✅ Đã lưu dữ liệu thanh-hoa vào E:\Tài liệu học\2024-2\Lưu trữ và xử lý dữ liệu lớn\Data\thanh-hoa.csv



...

✅ Đã lưu dữ liệu x.-ba-luong vào E:\Tài liệu học\2024-2\Lưu trữ và xử lý dữ liệu lớn\Data\x.-ba-luong.csv


🌤️ Crawling weather data:  91%|█████████ | 10/11 [11:13:52<1:09:49, 4189.38s/it]

✅ Đã lưu dữ liệu vinh-yen vào E:\Tài liệu học\2024-2\Lưu trữ và xử lý dữ liệu lớn\Data\vinh-yen.csv


🌤️ Crawling weather data: 100%|██████████| 11/11 [11:39:47<00:00, 3817.06s/it]  

✅ Đã lưu dữ liệu vung-tau vào E:\Tài liệu học\2024-2\Lưu trữ và xử lý dữ liệu lớn\Data\vung-tau.csv

✅ Đã crawl thành công: 11/11
